# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Изучение данных

In [13]:
import pandas as pd
data = pd.read_csv('data.csv')
data.info()
data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод** 

<dl>После вызова info по датафрейму можно сказать, что типы данных соответствуют значениям в соответствующих колонках. Однако, для столбца days_employed больше бы подошли целочисленные значения. В том же столбце одновременно присутствуют неожиданно большие значения и отрицательные. В total_income слишком много знаков после запятой, что нехарактерно для записи финансовых значений. </dl>

<dl></dl>

Есть пропуски в столбцах `days_employed` и `total_income`, в равных значениях (2174), если пропуски взаимосвязаны, то так записаны безработные в датафрейме.

<dl></dl>

<dl>Для дальнейшей обработки дубликатов требуется привести регистр к одному виду и найти уникальные значения.</dl>

## Шаг 2. Предобработка данных

### Обработка пропусков

In [14]:
# код ревьюера
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [15]:
# код ревьюера - доля пропусков в доходе по источнику дохода
data_nan = data[data['total_income'].isna()]
data_nan['income_type'].value_counts(normalize = True)

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

In [16]:
# Проверка теории, что количество пропусков в столбцах days_employed и total_income взаимосвязано.
display(len(data[data['total_income'].isna() & data['days_employed'].isna()])) 

# берем модуль от столбца days_employed, чтобы избавиться от отрицательных значений.
data['days_employed'] = data['days_employed'].apply(abs) 

# приводим большие значения к общей записи
data.loc[data['days_employed'] >20000, 'days_employed'] = data.loc[data['days_employed'] >20000, 'days_employed'] / 24 

median_days =  data['days_employed'].median() # находим медианные значения для столбца days_employed.
median_income = data['total_income'].median() # аходим медианные значения для столбца total_income.

# заполняем пропуски медианой в столбце total_income.
data['total_income'] = data['total_income'].fillna(median_income) 
# заполняем пропуски медианой в столбце days_employed.
data['days_employed'] = data['days_employed'].fillna(median_days)  

data.head(10) # проверка, что не осталось отрицательных значений в days_employed.
len(data[data['total_income'].isna() & data['days_employed'].isna()]) # проверка, что все значения заполнены. 


2174

0

**Вывод** 


Идея об отсутствии информации об офицальном месте работы у заемщиков с пропущенным значениями по столбцам `days_employed` и `total_income` – подтвердилась с помощью метода isna().<dl></dl>
<dl>Нужно взять в расчет то, что доход у заемщиков так или иначе должен быть, иначе бы они не смогли бы взять кредит, соответственно есть и опыт работы. Такими заемщиками могут быть фрилансеры, студенты и пенсионеры.</dl>
<dl>Заполним пропуски в указанных столбцах медианными значениями, методом fillna(), чтобы сохранить натуральность выборки.</dl>
<dl>В столбце `days_employed` были выгружены значения из разных систем, в одной с отрицательным значением, в другой в часах. С помощью модуля и деления на 24 часа, все значения столбца приведены к общему виду.</dl>


### Замена типа данных

In [17]:
data['days_employed'] = data['days_employed'].astype('int')
data['children'] = data['children'].astype('int')
data['dob_years'] = data['dob_years'].astype('int')
data['education_id'] = data['education_id'].astype('int')
data['debt'] = data['debt'].astype('int')
data['family_status_id'] = data['family_status_id'].astype('int')
# округление до 2 знаков после запятой, по финансовой форме записи.
data['total_income'] = round(data['total_income'],2) 





data.info() # проверка типов данных столбцов


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Привели все типы данных к корректному виду, используя метод astype, так как он подходит для изменений типа данных в целом столбце. Количество дней привели к целочисленному значению, так как точность до часа не нужна для проведения анализа. Оставили вещественное значение в столбце total_income, округлив до 2-х знаков после запятой, что больше подходит для записи дохода.        

### Обработка дубликатов

In [18]:
for unique in data: # цикл для подсчета уникальных значений в каждом столбце
    print(data[unique].value_counts().sort_values())
    
list_one=['education', 'family_status']

for lower in data: # цикл для приведения к нижнему регистру столбцов из списка list_one
    if lower in list_one:
        data[lower] = data[lower].str.lower()
display(data['education'].value_counts().sort_values()) # проверка 
display(data['family_status'].value_counts().sort_values()) # проверка 

display(data[data['gender']=='XNA']) # проверка необходимости строки с неизвестным полом.

data['gender'] = data['gender'].replace('XNA','F')

data['children'] = data['children'].replace(20,2)
data['children'] = data['children'].replace(-1,0)
data['dob_years'] = data['dob_years'].replace(0,data['dob_years'].median())

display(data.duplicated().sum()) #проверка количества явных дубликатов
data = data.drop_duplicates().reset_index(drop=True) # чистка от явных дубликатов



 5         9
 4        41
-1        47
 20       76
 3       330
 2      2055
 1      4818
 0     14149
Name: children, dtype: int64
14330       1
6634        1
4588        1
16581       1
13946       1
         ... 
204        14
438        15
327        16
133        16
2194     2180
Name: days_employed, Length: 7787, dtype: int64
75      1
74      6
73      8
19     14
72     33
20     51
71     58
70     65
69     85
68     99
0     101
21    111
67    167
66    183
22    183
65    194
23    254
24    264
64    265
63    269
62    352
61    355
25    357
60    377
26    408
55    443
59    444
51    448
53    459
57    460
58    461
46    475
54    479
47    480
52    484
56    487
27    493
45    497
28    503
49    508
32    510
43    513
50    514
37    537
48    538
30    540
29    545
44    547
36    555
31    560
39    573
33    581
42    597
38    598
34    603
41    607
40    609
35    617
Name: dob_years, dtype: int64
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ            

ученая степень             6
начальное                282
неоконченное высшее      744
высшее                  5260
среднее                15233
Name: education, dtype: int64

вдовец / вдова             960
в разводе                 1195
не женат / не замужем     2813
гражданский брак          4177
женат / замужем          12380
Name: family_status, dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.16,покупка недвижимости


72

In [19]:
data['gender'].value_counts()

F    14174
M     7279
Name: gender, dtype: int64

In [20]:
data[data['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [21]:
data[data['income_type'] == 'пенсионер']['dob_years'].value_counts().sort_index().head(10)

22    1
24    1
26    2
27    3
28    1
31    1
32    3
33    2
34    3
35    1
Name: dob_years, dtype: int64

**Вывод** 

Для того, чтобы оценить явные и неявные дубликаты в датафрейме вернули уникальные значения каждого столбца путем создания цикла. Определен третий пол в столбце gender, так как строка значима для выборки, то заменяем XNA на женский пол, так как его в выборке существенно больше мужского.   <dl></dl>
В столбцах `education` и `family_status` датафрейма присутствует смешанный регистр, приводим его к низкому путем создания еще одного цикла. <dl></dl>
Обнаруженное значение 0 в возрасте, заменим на медианное, так как определить пропущенный возраст невозможно, но заемщик по определению не мб несовершеннолетним.  <dl></dl>
Допущена опечатка в столбце `children`, навряд ли мб 20 детей, да и ещё и не как частный случай, скорее всего это значение 2. Значение -1 скорее всего пришло из другой записи, где -1 означает 0. Меняем 20 на 2 и -1 на 0.
 <dl></dl>Также в датафрейме были явные дубликаты, которые мы устранили методом drop_duplicates().

### Лемматизация

In [22]:
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter

lemmas=[]
lemma= " ".join(data['purpose'].unique()) # объединим уникальные значения по столзаемщиковбцу purpose в строку
lemma=m.lemmatize(lemma) # лемматизируем уникальные значения и получаемые их словарные формы.
print(Counter(lemma)) # определим самые частые леммы из столбца purpose

frequent= ['недвижимость','автомобиль','образование','свадьба','жилье'] # выделим в отдельный список ключевые леммы

def lem(purpose): # собственная функция для замены лемм на общие по группам 
    
    if frequent[0] in m.lemmatize(purpose) or frequent[4] in m.lemmatize(purpose):
        return frequent[0] 
    if frequent[1] in m.lemmatize(purpose):
        return frequent[1]
    if frequent[2] in m.lemmatize(purpose):
        return frequent[2]
    if frequent[3] in m.lemmatize(purpose):
        return frequent[3]
data['purpose'] = data['purpose'].apply(lem) # применяем функицию lem к столбцу purpose.
display(data['purpose']) #проверка


Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


0        недвижимость
1          автомобиль
2        недвижимость
3         образование
4             свадьба
             ...     
21448    недвижимость
21449      автомобиль
21450    недвижимость
21451      автомобиль
21452      автомобиль
Name: purpose, Length: 21453, dtype: object

**Вывод** 

Скорее всего операторы при записе отмечали причины кредита по собственным шаблонам, из-за чего образовались дубликаты целей для взятия кредита.  <dl></dl>
Нужно почистить столбец `purpose` от неявных дубликатов, обнаруженных при возврате уникальных значений в прошлом пункте. Лемматизируем значения столбца и в ручную выбираем самые часто встречаемые. Заменяем в столбце значения на 4 ключевые группы целей: недвижимость, автомобиль, образование, свадьба.

### Категоризация данных

In [23]:
# выделяем значение столбца с ключами в отдельную таблицу
ed_data=data[['education','education_id']].drop_duplicates().set_index('education_id') 
# выделяем значение столбца с ключами в отдельную таблицу
family_data = data[['family_status','family_status_id']].drop_duplicates().set_index('family_status_id') 
display(ed_data)
display(family_data)

#создаем категории по столбцу total_income, для упрощения ориентирования по доходам заемщиков
def group_income(income): 
    if income >=500000:
        return 'очень высокий'
    if 500000>income>=200000:
        return 'высокий'
    if 200000>income >=80000:
        return 'средний'
    if income < 80000:
        return 'низкий'
data['type_income'] = data['total_income'].apply(group_income)
#display(data.head()) проверка

def child_quantity(child): #создаем категории по столбцу children, для упрощения ориентирования по количеству детей заемщиков
    if child >=1:
        return 'есть'
    return 'нет'
data['children_quantity'] = data['children'].apply(child_quantity)
# display(data.head()) проверка

#создаем категории по столбцу family_status, для упрощения ориентирования по семейному статусу заемщиков
def marriage(spouse): 
    if spouse == 'женат / замужем':
        return 'да'
    return 'нет'
data['marriage'] = data['family_status'].apply(marriage)
data.head()


,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,не женат / не замужем


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,type_income,children_quantity,marriage
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,недвижимость,высокий,есть,да
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,автомобиль,средний,есть,да
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,недвижимость,средний,нет,да
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,образование,высокий,есть,да
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,свадьба,средний,нет,нет


**Вывод**

Выделели отдельно две таблицы с ключами и значениями по столбцам `family_status` и `education_id`, в дальнейшем чтобы не загромождать таблицу, эти значения можно будет удалить из исходного датафрейма, оставив ключи к ним. <dl></dl>
Для ответа на целевой вопрос по зависимости уровня дохода и возврата кредита в срок, поделили заемщиков по группам в зависимости от их дохода с оглядкой на медианное значение (145017.94): очень высокий (>500000 руб.), высокий (200000 руб.<доход<500000 руб.), средний (80000 руб.<доход<200000 руб.), низкий (< 80000 руб.). Данные значения добавлены в столбец `type_income`. <dl></dl>
 <dl></dl>Отдельно выделим столбцы по количеству детей (children_quantity) и наличию официального семейного статуса (marriage), чтобы посмотреть возможен ли ответ на целевые вопросы по более широким категориям, чем заданные в датафрейме.         

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [24]:
children_grouped = data.groupby('children').agg({'debt':'count'}) # сделаем сводную таблицу по столбцу children и debt, выделим общее количество в столбце debt заемщиков по категориям 
children_grouped['converse'] = data.groupby('children').agg({'debt':'sum'})/data.groupby('children').agg({'debt':'count'})  # получили конверсию путем деления количества долгов на количество должников
display(children_grouped.sort_values(by='converse')) # отсортируем по возрастанию просроченных кредитов

children_grouped_total = data.groupby('children_quantity').agg({'debt':'count'}) # сделаем сводную таблицу по столбцу children_quantity и debt, выделим общее количество в столбце debt
children_grouped_total['converse'] =  data.groupby('children_quantity').agg({'debt':'sum'})/data.groupby('children_quantity').agg({'debt':'count'}) # получили конверсию путем деления количества долгов на количество должников.
children_grouped_total.sort_values(by='converse') # отсортируем по возрастанию просроченных кредитов

,debt,converse
children,,
5,9,0.000000
0,14137,0.075263
3,330,0.081818
1,4808,0.092346
2,2128,0.094925
4,41,0.097561


,debt,converse
children_quantity,,
нет,14137,0.075263
есть,7316,0.092537


**Вывод**

<dl>Самый низкий показатель конверсии был у группы с 5 детьми, однако данная подгруппа слишком мала, чтобы повлиять на группу заемщиков с детьми. В общем, заемщики с детьми отдавали кредиты хуже, чем без детей. </dl>
<dl>Предполагаю, что наличие детей довольно часто приводит к неожиданным финансовым тратам или понижению среднемесячного заработка (больничный по уходу за ребенком по сниженной ставке), которые могут сдвинуть срок возврата средств кредита. </dl>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
family_grouped = data.groupby('family_status').agg({'debt':'count'}) # сделаем сводную таблицу по столбцу family_status и debt, выделим общее количество в столбце debt заемщиков по категориям 
family_grouped['converse'] = data.groupby('family_status').agg({'debt':'sum'})/data.groupby('family_status').agg({'debt':'count'})  # получили конверсию путем деления количества долгов на количество должников
display(family_grouped.sort_values(by='converse')) # отсортируем по возрастанию просроченных кредитов

family_grouped_total = data.groupby('marriage').agg({'debt':'count'}) # сделаем сводную таблицу по столбцу marriage и debt, выделим общее количество в столбце debt заемщиков по категориям 
family_grouped_total['converse'] =  data.groupby('marriage').agg({'debt':'sum'})/data.groupby('marriage').agg({'debt':'count'}) # получили конверсию путем деления количества долгов на количество должников
family_grouped_total.sort_values(by='converse') # отсортируем по возрастанию просроченных кредитов

,debt,converse
family_status,,
вдовец / вдова,959,0.065693
в разводе,1195,0.071130
женат / замужем,12339,0.075452
гражданский брак,4150,0.093494
не женат / не замужем,2810,0.097509


,debt,converse
marriage,,
да,12339,0.075452
нет,9114,0.088874


**Вывод**

Брак положительно отразился на отдачу кредита вовремя. Выражено это тем, что конверсия по тем заемщикам, которые состоят в официальных отношениях ниже, чем тех кто не имеет официального партнера. Возможно это связано с тем, что супруги достаточно часто выступают поручителями по кредиту и заинтересованы в отдаче кредита вовремя заемщиком.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
income_grouped_total = data.groupby('type_income').agg({'debt':'count'}) # сделаем сводную таблицу по столбцу type_income и debt, выделим общее количество в столбце debt заемщиков по категориям
income_grouped_total['converse'] =  data.groupby('type_income').agg({'debt':'sum'})/data.groupby('type_income').agg({'debt':'count'}) # получили конверсию путем деления количества долгов на количество должников
income_grouped_total.sort_values(by='converse') # отсортируем по возрастанию просроченных кредитов

,debt,converse
type_income,,
очень высокий,222,0.063063
высокий,4844,0.071016
низкий,2276,0.076450
средний,14111,0.085678


**Вывод**

Заемщики с большим доходом кредиты отдают лучше. Большие доходы дают большую финансовую подушку, а также имущество, которым в случае чп мб погасить свои задолженности, с меньшим доходом, такие маневры тяжелее проделать. 

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
purporse_grouped_total = data.groupby('purpose').agg({'debt':'count'}) # сделаем сводную таблицу по столбцу purpose и debt, выделим общее количество в столбце debt заемщиков по категориям.
purporse_grouped_total['converse'] =  data.groupby('purpose').agg({'debt':'sum'})/data.groupby('purpose').agg({'debt':'count'}) # получили конверсию путем деления количества долгов на количество должников.
purporse_grouped_total.sort_values(by='converse') # отсортируем по возрастанию просроченных кредитов

,debt,converse
purpose,,
недвижимость,10811,0.072334
свадьба,2323,0.080069
образование,4013,0.092200
автомобиль,4306,0.093590


**Вывод**

Хуже всего отдают кредиты за машины (конверсия 0.093590), а лучший показатель у недвижимости (конверсия 0.072334). Сказывается стресс, так как при невозможности выплаты кредита за машину, место для сна никуда не пропадает, что не сказать про задолженности по ипотеке.

## Шаг 4. Общий вывод

<dl>При изучении базы данных в ней были обнаружены ошибки и опечатки, которые бы значительно повлияли на конечный результат анализа. </dl>
После устранения дубликатов и заполнения пропусков, в общем, приведении датафрейма к пригодном для исследования виду, стали известны следующие рекомендации для банка исследования: Лучше всего выдавать кредиты заемщикам, состоящим в официальных отношениях без детей, имеющих высокий доход и предполагающих покупку жилья для себя или для бизнеса. Данная рекомендация не означает отказ в кредитах другим группам потенциальных заемщиков, а лишь предполагает упрощение получения одобрения кредита для вышеназванной группы.  